In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import numpy as np
import time
from tqdm import trange
import matplotlib.pyplot as plt

In [3]:
import gym
import pybullet as p
import stage.envs
from stage.tasks.kuka import KukaPETS
from stage.utils.nn import use_gpu
use_gpu()

In [4]:
savepath = parentdir + '/data/kuka/'

In [5]:

kuka = KukaPETS(render=False, 
                   action_parameterization='pd')
nq, nv, nu, nx, na = kuka.nq, kuka.nv, kuka.nu, kuka.nx, kuka.na

In [ ]:
_ = kuka.learn(100, verbose=True)
kuka.save_training_data(savepath+'data_pd')

Network training: 100%|██████████| 20/20 [00:01<00:00, 15.74epoch(s)/s, Training loss MSE=205.14005]


Iteration:  0
avg. decision time:  0.004452163378397623
obs. reward:  -733.5325
act. reward:  -5.674615


Network training: 100%|██████████| 20/20 [00:02<00:00,  8.33epoch(s)/s, Training loss MSE=113.69343] 


Iteration:  1
avg. decision time:  0.190067990620931
obs. reward:  -502.60223
act. reward:  -1.1632756


Network training: 100%|██████████| 20/20 [00:03<00:00,  5.57epoch(s)/s, Training loss MSE=89.47478] 


...

Iteration:  10
avg. decision time:  0.1925849405924479
obs. reward:  -883.94226
act. reward:  -1.4398832


Network training: 100%|██████████| 20/20 [00:13<00:00,  1.51epoch(s)/s, Training loss MSE=40.173508]


Iteration:  11
avg. decision time:  0.1926958990097046
obs. reward:  -913.16
act. reward:  -1.5899272


Network training: 100%|██████████| 20/20 [00:14<00:00,  1.41epoch(s)/s, Training loss MSE=59.05766] 


Iteration:  12
avg. decision time:  0.19254347483317058
obs. reward:  -2624.578
act. reward:  -18.185572


In [ ]:
# traj, log = kuka.perform(kuka.goal)
# act_seq = traj[:, 14:28]
# initial_obs = traj[0, :14]
# final_obs = traj[-1, :14]

# n_sample = 50
# obs_traj = kuka.dynamics.unroll(initial_obs, 
#                                       act_seq, 
#                                       n_sample)

# predicted_err = torch.norm(obs_traj[-1, :, :]-final_obs.expand(n_sample, -1), p=2, dim=1)
# print (predicted_err.mean())
# print (predicted_err.std())

